In [ ]:
import os
# Set the correct path to https://github.com/PruneTruong/DenseMatching repository on your filesystem
module_path = os.path.abspath(os.path.join('..'))
# =======================================================================================++ #

In [111]:
import argparse
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import torch
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import sys
import matplotlib.cm as cm
import os.path as os

if module_path not in sys.path:
    sys.path.append(module_path)
from model_selection import model_type, pre_trained_model_types, select_model
from datasets.util import pad_to_same_shape
torch.set_grad_enabled(False)
from utils_flow.pixel_wise_mapping import remap_using_flow_fields
from utils_flow.visualization_utils import overlay_semantic_mask, make_sparse_matching_plot
from utils_flow.util_optical_flow import flow_to_image  
from models.inference_utils import estimate_mask
from utils_flow.flow_and_mapping_operations import convert_flow_to_mapping
from validation.utils import matches_from_flow
from admin.stats import DotDict 
%matplotlib inline

# Choose and load the correct model for dense matching. For example, PDCNet+megadepth.  

In [112]:
# choose model 
model = 'PDCNet'
pre_trained_model = 'megadepth'
flipping_condition = False 
global_optim_iter = 3
local_optim_iter = 7 
path_to_pre_trained_models = osp.join(module_path,'assets/pre_trained_models/')
    
if model not in model_type:
    raise ValueError('The model that you chose is not valid: {}'.format(model))
if pre_trained_model not in pre_trained_model_types:
    raise ValueError('The pre-trained model type that you chose is not valid: {}'.format(pre_trained_model))


# inference parameters for PDC-Net
network_type = model  # will only use these arguments if the network_type is 'PDCNet' or 'PDCNet_plus'
choices_for_multi_stage_types = ['d', 'h', 'ms']
multi_stage_type = 'h'
if multi_stage_type not in choices_for_multi_stage_types:
    raise ValueError('The inference mode that you chose is not valid: {}'.format(multi_stage_type))

confidence_map_R =1.0
ransac_thresh = 1.0
mask_type = 'proba_interval_1_above_10'  # for internal homo estimation
homography_visibility_mask = True
scaling_factors = [0.5, 0.6, 0.88, 1, 1.33, 1.66, 2]
compute_cyclic_consistency_error = True  # here to compare multiple uncertainty 

# usually from argparse
args = DotDict({'network_type': network_type, 'multi_stage_type': multi_stage_type, 'confidence_map_R': confidence_map_R, 
                'ransac_thresh': ransac_thresh, 'mask_type': mask_type, 
                'homography_visibility_mask': homography_visibility_mask, 'scaling_factors': scaling_factors, 
                'compute_cyclic_consistency_error': compute_cyclic_consistency_error})


In [113]:
# define network and load network weights
network, estimate_uncertainty = select_model(
    model, pre_trained_model, args, global_optim_iter, local_optim_iter,
    path_to_pre_trained_models=path_to_pre_trained_models)
estimate_uncertainty = True  
# here, we overwrite it, to also estimate uncertainty according to forward-backward for networks that do not predict a confidence measure

Model: PDCNet
Pre-trained-model: megadepth
GOCor: Local iter 7
GOCor: Global iter 3


/home/klaus/anaconda3/envs/dense_matching_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/klaus/anaconda3/envs/dense_matching_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


../assets/pre_trained_models/PDCNet_megadepth.pth.tar


## Set correct directories for KDD project

In [5]:
import os
import os.path as osp
def find_root_folder(starting_path = os.getcwd()):
    from os.path import isfile, join, dirname
    path = starting_path
    i = 0
    found = False
    while i < 100:
        found = ('LICENSE' in [f for f in os.listdir(path) if isfile(join(path, f))])
        if found:
            break
        else:
            i += 1
            path = dirname(path)            
    if found:
        print(f"Found directory with license at {path}")
    else:
        raise FileNotFoundError("Could not find LICENSE file in ancestral directory")
    return path
ROOT_FOLDER = find_root_folder()
DS_FOLDER = osp.join(ROOT_FOLDER,'kddbr-2022')


Found directory with license at /home/klaus/eclipse_draft


'/home/klaus/eclipse_draft'

## 

In [174]:
import os
import os.path as osp
import pandas as pd
from tqdm import tqdm, trange

PLOT = False
df = pd.read_csv(osp.join(DS_FOLDER,'public.csv'))
df['path'] = df['Filename']
# We map the filenames on public.csv to the actual path.
# to the training image folder
df.loc[~pd.isna(df['North']), 'path'] = df.loc[~pd.isna(df['North']), 'path'].apply(lambda x: osp.join(DS_FOLDER, 'train','train', x))
# to the test images folder
df.loc[pd.isna(df['North']), 'path'] = df.loc[pd.isna(df['North']), 'path'].apply(lambda x: osp.join(DS_FOLDER, 'test','test', x))


def get_images(which_id):
    """ Returns the image corresponding to the ``which_id``-th row of the dataframe"""
    img = imageio.imread(df['path'].values[which_id], pilmode='RGB')
    query_image, reference_image = img[:,:120,:], img[:,120:,:] 
    query_image_shape = query_image.shape
    ref_image_shape = reference_image.shape
    return query_image, reference_image

"""
    We create a list L of arrays, each array containing information extracted by the dense matching model.
"""
L = []

for which_id in trange(df.shape[0]):
    query_image, reference_image = get_images(which_id)
    
    """ 
        Prepare query and reference images. 
    """
    # convert the images to correct format to be processed by the network: torch Tensors, format B, C, H, W. 
    # pad both images to the same size, to be processed by network
    query_image_, reference_image_ = pad_to_same_shape(query_image, reference_image)

    # convert numpy to torch tensor and put it in right format
    query_image_ = torch.from_numpy(query_image_).permute(2, 0, 1).unsqueeze(0)
    reference_image_ = torch.from_numpy(reference_image_).permute(2, 0, 1).unsqueeze(0)
    """ 
        Estimate flow / uncertainty using the given model. 
    """
    if estimate_uncertainty:
        estimated_flow, uncertainty_components = network.estimate_flow_and_confidence_map(query_image_, reference_image_)
    else:
        if args.flipping_condition and 'GLUNet' in args.model:
            estimated_flow = network.estimate_flow_with_flipping_condition(query_image_, reference_image_,
                                                                           mode='channel_first')
        else:
            estimated_flow = network.estimate_flow(query_image_, reference_image_, mode='channel_first')
    # removes the padding
    estimated_flow = estimated_flow[:, :, :ref_image_shape[0], :ref_image_shape[1]]

    # convert to numpy and reformat
    estimated_flow_numpy = estimated_flow.squeeze().permute(1, 2, 0).cpu().numpy()

    # warp the query image according to the estimated flow
    warped_query_image = remap_using_flow_fields(query_image, estimated_flow_numpy[:, :, 0],
                                                 estimated_flow_numpy[:, :, 1]).astype(np.uint8)
    alpha = 0.5
    img_warped_overlay_on_target_masked = warped_query_image * alpha + reference_image * alpha
    # confidence estimation + visualization
    if not estimate_uncertainty: 
        raise ValueError
    uncertainty_key = 'p_r'  # 'inv_cyclic_consistency_error' 
    #'p_r', 'inv_cyclic_consistency_error' can also be used as a confidence measure
    # 'cyclic_consistency_error' can also be used, but that's an uncertainty measure
    min_confidence = 0.30
    confidence_map = uncertainty_components[uncertainty_key]
    confidence_map = confidence_map[:, :, :ref_image_shape[0], :ref_image_shape[1]]
    confidence_map_numpy = confidence_map.squeeze().detach().cpu().numpy()
    
    color = [255, 102, 51]
    confidence_map_numpy = confidence_map.squeeze().detach().cpu().numpy()
    confident_mask = (confidence_map_numpy > min_confidence).astype(np.uint8)
    confident_warped = overlay_semantic_mask(warped_query_image, ann=255 - confident_mask*255, color=color)
    # get the mask according to uncertainty estimation
    mask_type = 'proba_interval_1_above_10' # 'cyclic_consistency_error_below_2' 
    mask_padded = estimate_mask(mask_type, uncertainty_components) 
    if 'warping_mask' in list(uncertainty_components.keys()):
        # get mask from internal multi stage alignment, if it took place
        mask_padded = uncertainty_components['warping_mask'] * mask_padded
    # remove the padding
    mask = mask_padded[:, :ref_image_shape[0], :ref_image_shape[1]]
    # remove point that lead to outside the query image
    mapping_estimated = convert_flow_to_mapping(estimated_flow)
    mask = mask & mapping_estimated[:, 0].ge(0) & mapping_estimated[:, 1].ge(0) & \
    mapping_estimated[:, 0].le(query_image_shape[1] - 1) & mapping_estimated[:, 1].le(query_image_shape[0] - 1)

    mkpts_query, mkpts_ref = matches_from_flow(estimated_flow, mask)

    confidence_values = confidence_map.squeeze()[mask.squeeze()].cpu().numpy()
    sort_index = np.argsort(np.array(confidence_values)).tolist()[::-1]  # from highest to smallest
    confidence_values = np.array(confidence_values)[sort_index]
    mkpts_query = np.array(mkpts_query)[sort_index]
    mkpts_ref = np.array(mkpts_ref)[sort_index]

    if len(mkpts_query) > 5:
        """ Use OpenCV to find the best homography transformation,
            given the marker points obtained from dense matching.
        """
        M, mask = cv.findHomography(mkpts_query, mkpts_ref, cv.RANSAC,5.0)
    else:
        M = np.zeros((3,3))
    
    """ Create feature vector."""
    confidence_map_numpy = confidence_map_numpy.reshape(-1)
    res = np.zeros(20)
    res[:9] = M.reshape(-1)
    res[9] = len(mkpts_query)
    res[10] = confidence_map_numpy.reshape(-1).mean()
    res[11] = estimated_flow_numpy[:,:,0].mean()
    res[12] = estimated_flow_numpy[:,:,0].std()
    res[13] = estimated_flow_numpy[:,:,1].mean()
    res[14] = estimated_flow_numpy[:,:,1].std()
    res[15:20] = np.quantile(confidence_map_numpy,[0,0.1,0.25,0.5,1])
    
    # Append feature vector to list
    L.append(res[None,:])


100%|█████████████████████████████████| 146262/146262 [6:17:02<00:00,  6.47it/s]


## We concatenate all the arrays from the list L into a big array X. From it, we create a feature dataframe and save it for (training + validation) and test subsets.


In [178]:

X = np.concatenate(L,axis=0)
col = [f'mat_{i}' for i in range(9)] + ['n_pts'] + ['cm_mean','ef0_mean','ef0_std','ef1_mean','ef1_std'] +  [f'cm_q{i}' for i in range(5)]
feat_df = pd.DataFrame(X,index=range(X.shape[0]),columns=col)
display(feat_df)

# Define and create output folder if it does not exist
OUT_FOLDER  = osp.join(ROOT_FOLDER,'features', model + '-' + pre_trained_model) 
os.makedirs(OUT_FOLDER, exist_ok  = True)

feat_df.loc[~pd.isna(df['North']), :].to_csv(osp.join(OUT_FOLDER,'trainval.csv'),index=False)
feat_df.loc[pd.isna(df['North']), :].to_csv(osp.join(OUT_FOLDER,'test.csv'),index=False)


146262


,mat_0,mat_1,mat_2,mat_3,mat_4,mat_5,mat_6,mat_7,mat_8,n_pts,cm_mean,ef0_mean,ef0_std,ef1_mean,ef1_std,cm_q0,cm_q1,cm_q2,cm_q3,cm_q4
0,1.002423,-0.001384,-2.929688,0.000169,1.001043,-1.337759,0.000001,4.164572e-06,1.0,13764.0,0.572765,2.888732,0.118693,1.277729,0.083238,0.464305,0.572845,0.572869,0.572871,0.572872
1,0.998573,0.001190,-0.353788,0.001521,0.997956,0.848945,-0.000023,4.255099e-07,1.0,14096.0,0.567747,0.262010,0.151464,-0.902149,0.223343,0.174298,0.568731,0.572271,0.572810,0.572872
2,0.996697,0.001945,1.621848,-0.002061,1.000195,-3.803263,-0.000034,2.240335e-05,1.0,13578.0,0.572625,-1.623801,0.073526,3.904920,0.115052,0.390947,0.572856,0.572867,0.572870,0.572872
3,0.981403,-0.010081,3.957249,0.010763,0.995305,-6.589612,0.000009,-6.674789e-05,1.0,13107.0,0.572373,-2.456800,0.589099,5.979247,0.431804,0.453649,0.572844,0.572870,0.572872,0.572872
4,1.003583,0.002621,-8.319884,0.001843,1.001703,-0.813738,0.000041,-3.694162e-05,1.0,13196.0,0.571685,8.009210,0.303784,0.571931,0.147131,0.362815,0.572329,0.572865,0.572870,0.572872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146257,1.037009,0.026269,-10.391671,-0.013865,0.993531,10.232949,0.000061,6.216618e-05,1.0,12482.0,0.571941,7.084321,1.155725,-8.482187,0.845956,0.343910,0.572322,0.572844,0.572868,0.572872
146258,1.009781,-0.007256,-11.139119,0.000272,0.999674,-0.489926,-0.000030,8.783195e-05,1.0,12732.0,0.572101,11.016906,0.601743,0.772840,0.306503,0.489332,0.572452,0.572865,0.572870,0.572872
146259,1.004349,-0.005489,-2.613301,0.000747,0.998082,3.039214,0.000044,-5.560081e-05,1.0,13451.0,0.572544,2.684564,0.147633,-3.073310,0.159691,0.330920,0.572717,0.572862,0.572869,0.572872
146260,0.986190,0.000395,7.049865,0.003211,0.997465,-2.680462,-0.000026,2.011234e-06,1.0,13031.0,0.572071,-6.440472,0.389240,2.600944,0.188957,0.385985,0.572845,0.572867,0.572871,0.572872
